# 2. Which DNA Patterns Play The Role of Molecular Clocks? (Part 2)

```
관찰된 데이터셋에서 어떤 이벤트는 확률이 0은 아니지만 발생하지 않을 가능성이 있다.
관측된 발생빈도는 0 이지만 이 이벤트의 경험적 확률을 0으로 설정하면 문제가 될 수 있다.
이런 경우에는 확률을 인위적으로 조정하여 문제를 완화시킬 수 있다.

pseudocounts 라고 불리는 작은 숫자로 0을 대체 (Laplace’s Rule of Succession)
```

In [1]:
def CountMatrix(Motifs):
    t = len(Motifs)      # DNA string 갯수
    k = len(Motifs[0])   # 각 string의 길이
    
    count = {'A':[0]*k, 'C':[0]*k, 'G':[0]*k,'T':[0]*k}

    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    
    return count

In [2]:
def CountWithPseudocounts(Motifs):
    # adds 1 to each element of Count(Motifs)
    
    t = len(Motifs)
    k = len(Motifs[0])
    
    count = {'A':[1]*k, 'C':[1]*k, 'G':[1]*k,'T':[1]*k}  # Pseudocounts

    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    
    return count

In [3]:
strings = ['AACGTA', 
           'CCCGTT', 
           'CACCTT', 
           'GGATTA', 
           'TTCCGG']

In [4]:
CountMatrix(strings)

{'A': [1, 2, 1, 0, 0, 2],
 'C': [2, 1, 4, 2, 0, 0],
 'G': [1, 1, 0, 2, 1, 1],
 'T': [1, 1, 0, 1, 4, 2]}

In [5]:
CountWithPseudocounts(strings)

{'A': [2, 3, 2, 1, 1, 3],
 'C': [3, 2, 5, 3, 1, 1],
 'G': [2, 2, 1, 3, 2, 2],
 'T': [2, 2, 1, 2, 5, 3]}

In [6]:
def ProfileWithPseudocounts(Motifs):
    profile = CountWithPseudocounts(Motifs)
    
    m = 0
    for key, val in profile.items():
        m += val[0]
    # print(m)
    
    for key, val in profile.items():
        val[:] = [x / m for x in val]
        
    return profile

In [7]:
ProfileWithPseudocounts(strings)

{'A': [0.2222222222222222,
  0.3333333333333333,
  0.2222222222222222,
  0.1111111111111111,
  0.1111111111111111,
  0.3333333333333333],
 'C': [0.3333333333333333,
  0.2222222222222222,
  0.5555555555555556,
  0.3333333333333333,
  0.1111111111111111,
  0.1111111111111111],
 'G': [0.2222222222222222,
  0.2222222222222222,
  0.1111111111111111,
  0.3333333333333333,
  0.2222222222222222,
  0.2222222222222222],
 'T': [0.2222222222222222,
  0.2222222222222222,
  0.1111111111111111,
  0.2222222222222222,
  0.5555555555555556,
  0.3333333333333333]}

In [8]:
def ConsensusMatrix(Motifs):
    t = len(Motifs)      # DNA string 갯수
    k = len(Motifs[0])   # 각 string의 길이
    
    count_matrix = {'A':[0]*k, 'C':[0]*k, 'G':[0]*k,'T':[0]*k}

    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count_matrix[symbol][j] += 1

    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        
        for symbol in "ACGT":
            if count_matrix[symbol][j] > m:
                m = count_matrix[symbol][j]
                frequentSymbol = symbol
                
        consensus += frequentSymbol
        
    return consensus

def HammingDistance(p, q):
    t_len = max(len(p), len(q))
    ham = [1 for x in range(t_len) if p[x] != q[x]]
    count = sum(ham)
    return count

def Score(Motifs):
    consensus = ConsensusMatrix(Motifs)
    
    score = 0
    for motif in Motifs:
        score += HammingDistance(consensus, motif)
    
    return score

def ProbMotif(Text, Profile):
    prob = 1
    for index, nucleotide in enumerate(Text):
        prob *= Profile[nucleotide][index]
    
    return prob

def ProfileMostProbableKmer(text, k, profile):
    p = -1
    result = text[0:k]
    for i in range(len(text)-k+1):
        seq = text[i:i+k]        
        pr = ProbMotif(seq, profile)
        if pr > p:
            p = pr
            result = seq
    return result

def window(s, k):
    for i in range(1 + len(s) - k):
        yield s[i:i+k]
        
def GreedyMotifSearchWithPseudocounts(Dna, k, t):
    bestMotifs = []
    bestScore = 0   
    
    # t개의 dna에서 각각 첫번째 k-mer 추출
    for string in Dna:
        bestMotifs.append(string[:k])
        
    bestScore = Score(bestMotifs)
    
    base = Dna[0]
    n = len(base)
    
    # base : 첫번째 dna에서 모든 k-mer 추출
    for i in window(base, k):
        newMotifs = [i]

        # iterate over len(DNA)
        for j in range(1, t):
            # build up motifs and build profile using them.
            profile = ProfileWithPseudocounts(newMotifs[:j])
            pattern = ProfileMostProbableKmer(Dna[j], k, profile)
            newMotifs.append(pattern)
        
        currentScore = Score(newMotifs)
        if k == 3:
            print(currentScore, newMotifs)
        
        if currentScore < bestScore:
            bestScore = currentScore
            bestMotifs = newMotifs
            
    return bestMotifs

In [9]:
Dna = [
    "GGCGTTCAGGCA",
    "AAGAATCAGTCA",
    "CAAGGAGTTCGC",
    "CACGTCAATCAC",
    "CAATAATATTCG"
]

GreedyMotifSearchWithPseudocounts(Dna, 3, len(Dna))

4 ['GGC', 'GTC', 'GGA', 'GTC', 'TTC']
4 ['GCG', 'AAG', 'AAG', 'ACG', 'TCG']
3 ['CGT', 'AGT', 'AGT', 'CGT', 'AAT']
3 ['GTT', 'GTC', 'GTT', 'GTC', 'ATT']
2 ['TTC', 'ATC', 'TTC', 'ATC', 'TTC']
2 ['TCA', 'TCA', 'TCG', 'TCA', 'TCG']
2 ['CAG', 'CAG', 'CAA', 'CAA', 'CAA']
3 ['AGG', 'AAG', 'AAG', 'ACG', 'AAT']
4 ['GGC', 'GTC', 'GGA', 'GTC', 'TTC']
5 ['GCA', 'GAA', 'GGA', 'TCA', 'TAA']


['TTC', 'ATC', 'TTC', 'ATC', 'TTC']

결핵을 일으키는 Mycobacterium tuberculosis bacterium (MTB)의 유전자 중 regulatory motif 찾기

In [10]:
Dna = ["GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC", 
       "CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG", 
       "ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC", 
       "GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC", 
       "GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG", 
       "CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA", 
       "GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA", 
       "GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG", 
       "GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG", 
       "TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"]

In [11]:
k = 15
t = len(Dna)

motifs = GreedyMotifSearchWithPseudocounts(Dna, k, t)
print(motifs)
print(Score(motifs))

['GGACTTCAGGCCCTA', 'GGTCAAACGACCCTA', 'GGACGTAAGTCCCTA', 'GGATTACCGACCGCA', 'GGCCGAACGACCCTA', 'GGACCTTCGGCCCCA', 'GGACTTCTGTCCCTA', 'GGACTTTCGGCCCTG', 'GGACTAACGGCCCTC', 'GGACCGAAGTCCCCG']
35


```
Week_3의 GreedyMotifSearch 결과의 Score = 64

Week_4의 GreedyMotifSearchWithPseudocounts 결과의 Score = 35  로 크게 향상되었음.

```

### Monte Carlo algorithms (Randomized algorithms)

```
DNA에서 무작위로 선택한 k-mer 모음, 이를 이용해 Profile 구성하고, 이 Profile을 사용해 새로운 k-mer 모음을 생성. 
두번째로 생성한 k-mer의 score가 이전 것보다 더 좋음.
k-mer의 score를 계속 향상되는 한 이 과정을 계속 반복.

```

In [12]:
import random

In [13]:
# Profile을 이용해 motifs 추출
def Motifs(Profile, Dna):
    motifs = []
    
    k = len(Profile['A'])
    
    for string in Dna:
        motif = ProfileMostProbableKmer(string, k, Profile)
        motifs.append(motif)
        
    return motifs

In [14]:
Dna = [
    "TTACCTTAAC",
    "GATGTCTGTC",
    "ACGGCGTTAG",
    "CCCTAACGAG",
    "CGTCAGAGGT"
]

profile = {
    'A': [0.8, 0.0, 0.0, 0.2],
    'C': [0.0, 0.6, 0.2, 0.0],
    'G': [0.2, 0.2, 0.8, 0.0],
    'T': [0.0, 0.2, 0.0, 0.8]
}

Motifs(profile, Dna)

['ACCT', 'ATGT', 'GCGT', 'ACGA', 'AGGT']

In [15]:
def RandomMotifs(Dna, k, t):
    randomMotif =[]
    for i in range(t):
        r = random.randint(0, len(Dna[0])-k) # random.randint(0, m) : 0 ~ m 사이의 정수를 랜덤으로 선택
        randomMotif.append(Dna[i][r:r+k])
        
    return randomMotif

In [16]:
Dna = [
    "TTACCTTAAC",
    "GATGTCTGTC",
    "ACGGCGTTAG",
    "CCCTAACGAG",
    "CGTCAGAGGT"
]

RandomMotifs(Dna, 3, len(Dna))

['ACC', 'ATG', 'TAG', 'TAA', 'GTC']

In [17]:
def RandomizedMotifSearch(Dna, k, t):

    # 최초에 random motif 생성
    newMotifs = RandomMotifs(Dna, k, t)
    bestMotifs = newMotifs
    bestScore = Score(newMotifs)
    
    # random 추출 반복
    while True:
        profile = ProfileWithPseudocounts(newMotifs)       
        newMotifs = Motifs(profile, Dna)
        
        currentScore = Score(newMotifs)
        
        if currentScore < bestScore:
            bestScore = currentScore
            bestMotifs = newMotifs
        else:
            return bestMotifs 
            
    return bestMotifs

In [18]:
k = 3
t = len(Dna)

RandomizedMotifSearch(Dna, k, t)

['CCT', 'CTG', 'ACG', 'CCC', 'CAG']

In [19]:
Dna = [
    "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
    "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
    "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
    "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
    "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"
]

k = 8
t = len(Dna)

motifs = RandomizedMotifSearch(Dna, k, t)

print(motifs)
print(Score(motifs))

['CCCCTCTC', 'AAGGTGCC', 'CCGAAAGA', 'ACGTCGGT', 'CCACCAGC']
18


### Gibbs Sampling

```
결합 분포가 알려져 있지 않으나 각 변수의 조건부 확률분포는 알려져 있을 경우 이에 의존하여 교대로 표본을 채취하는 방법.
특히 MCMC (Markov Chain Monte Carlo) 방법에서 target distribution의 추정에 매우 중요한 역할을 한다.
https://m.blog.naver.com/sw4r/221917843395

RandomizedMotifSearch 는 한 번의 iteration에서 t 개의 모든 string을 변경한다.
이것은 올바른 motif가 다음 iteration에서는 무시될 가능성이 있다.

GibbsSampler는 한 번의 iteration에서 1개의 string만 변경하고 나머지는 유지한다.
선택된 각 k-mer들은 profile에 의해 계산된 확률이 다를 수 있다.

GibbsSampler는 k-mer 중에서 삭제할 (무시할) string을 무작위로 선택한다.
나머시 t-1 개의 k-mer로 motif, count, profile matrix를 생성한다. 여기에 pseudocounts를 적용한다.
pseudocounts를 적용하면 확률의 합이 1이 되지 않기 때문에 이를 Normalize 할 필요가 있다.
```

In [20]:
def Normalize(Probabilities):
    total_p = sum(Probabilities.values())
    p = [(k, v/total_p) for k, v in Probabilities.items()]
    norm_p = dict(p)
    return norm_p

In [21]:
prior_prob = {'A': 0.1, 'C': 0.1, 'G': 0.1, 'T': 0.1}

Normalize(prior_prob)

{'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}

In [22]:
def WeightedDie(Probabilities):
    p = random.uniform(0,1)
    #print(p)
    
    norm_p = Normalize(Probabilities)

    sum_p = 0
    for key,value in norm_p.items():
        sum_p = sum_p + value
        #print('sum_p :', sum_p)
        if p < sum_p:
            return key

In [23]:
selected_key = WeightedDie(prior_prob)
print(selected_key)

G


```
string에서 무작위로 k-mer를 샘플링하는 시뮬레이션
```

In [24]:
def ProfileGeneratedString(Text, profile, k):
    """
    profile 기반으로 Text에 대해 가능한 모든 k-mer에 대한 확률을 계산.
    이 확률을 Normalize 한 확률을 이용해 WeightedDie 추출
    """
    n = len(Text)
    probabilities = {}
    for i in range(0, n-k+1):
        probabilities[Text[i:i+k]] = ProbMotif(Text[i:i+k], profile)

    probabilities = Normalize(probabilities)
    
    return WeightedDie(probabilities)

In [25]:
string = 'AAACCCAAACCC'

# 2-mer 에 대한 사전확률
profile = {'A': [0.5, 0.1], 'C': [0.3, 0.2], 'G': [0.2, 0.4], 'T': [0.0, 0.3]}

ProfileGeneratedString(string, profile, 2)

'AC'

In [26]:
def GibbsSampler(Dna, k , t, N):
    M = RandomMotifs(Dna, k, t)
    BestMotifs = M
    for j in range(N):
        i = random.randint(0,t-1)
        M.pop(i)
        Profile =  ProfileWithPseudocounts(M)
        newMotif =  ProfileGeneratedString(Dna[i], Profile, k)
        M.insert(i,newMotif)
        if Score(M) < Score(BestMotifs):
            BestMotifs = M

    return BestMotifs

In [27]:
Dna = [
    "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
    "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
    "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
    "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
    "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"
]

k = 8
t = len(Dna)
N = 100

motifs = GibbsSampler(Dna, k, t, N)

print(motifs)
print(Score(motifs))

['AACGGCCA', 'AGGTGCCA', 'TAGTACCG', 'AAGTTTCA', 'AGCTCCAC']
13


In [28]:
Dna = [
    "GGCGTTCAGGCA",
    "AAGAATCAGTCA",
    "CAAGGAGTTCGC",
    "CACGTCAATCAC",
    "CAATAATATTCG"
]

k = 3
t = len(Dna)
N = 10

motifs = GibbsSampler(Dna, k, t, N)

print(motifs)
print(Score(motifs))

['CGT', 'TCA', 'AGG', 'CGT', 'CAA']
7


In [29]:
# Quiz

def Profile(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    
    profile = {'A':[0]*k, 'C':[0]*k, 'G':[0]*k,'T':[0]*k}

    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            profile[symbol][j] += 1
            
    for key, val in profile.items():
        val[:] = [x / t for x in val]
        
    return profile

Dna = [
    "ATGAGGTC",
    "GCCCTAGA",
    "AAATAGAT",
    "TTGTGCTA"
]

motifs = ['GTC', 
         'CCC', 
         'ATA', 
         'GCT']
profile = Profile(motifs)
# print(profile)

Motifs(profile, Dna)

['GTC', 'GCC', 'ATA', 'GCT']